In [ ]:
# Step 1: 필요한 라이브러리 설치
!pip install tensorflow pillow #tensorflow
!pip install --upgrade tensorflow
!pip install firebase-admin #google firebase

In [ ]:
# Step 2: TeachableMachine 모델 압축파일 업로드
from google.colab import files
uploaded = files.upload()  # .zip 파일 업로드

Saving converted_keras.zip to converted_keras.zip


In [ ]:
# Step 3: TeachableMachine 모델 압축파일 압축 해제
import zipfile
import os

zip_path = list(uploaded.keys())[0]
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('tm_model')

In [14]:
# Step 4: TeachableMachine 모델 로드

import tensorflow as tf

# Define a custom DepthwiseConv2D class that ignores the 'groups' argument
class CustomDepthwiseConv2D(tf.keras.layers.DepthwiseConv2D):
    def __init__(self, *args, **kwargs):
        # Remove the 'groups' argument if it exists
        if 'groups' in kwargs:
            del kwargs['groups']
        super().__init__(*args, **kwargs)

    def get_config(self):
        config = super().get_config()
        # Remove the 'groups' argument from the config
        if 'groups' in config:
            del config['groups']
        return config

# Load the model with the custom object
model = tf.keras.models.load_model('tm_model/keras_model.h5', custom_objects={'DepthwiseConv2D': CustomDepthwiseConv2D})
print("✅ 모델 로드 완료")

#import tensorflow as tf
#model = tf.keras.models.load_model('tm_model/keras_model.h5')
#print("✅ 모델 로드 완료")

✅ 모델 로드 완료


In [ ]:
# Step 5: 웹캠에서 이미지 캡처
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import PIL.Image
import io
import numpy as np

# 웹캠 실행 함수
def capture_image():
    # Request webcam permission
    display(Javascript('''
        async function requestCameraPermission() {
            try {
                await navigator.mediaDevices.getUserMedia({video: true});
                return true;
            } catch (error) {
                console.error('Error accessing webcam:', error);
                return false;
            }
        }
        requestCameraPermission();
    '''))

    # Check if permission was granted
    permission_granted = eval_js("requestCameraPermission()")
    if not permission_granted:
        print("Webcam permission denied. Please grant permission to use the webcam.")
        return None

    display(Javascript('''
        async function capture() {
            // 이미 있는 요소 제거
            const oldDiv = document.getElementById('capture-div');
            if (oldDiv) oldDiv.remove();
            const oldVideo = document.querySelector('video');
            if (oldVideo) oldVideo.remove();

            const div = document.createElement('div');
            div.id = 'capture-div';
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);
            document.body.appendChild(div);

            const video = document.createElement('video');
            video.style.display = 'block';
            document.body.appendChild(video);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            // Resize video to fit
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());

            const dataUrl = canvas.toDataURL('image/jpeg');
            div.remove();
            video.remove();
            return dataUrl;
        }
        capture();
    '''))

    data = eval_js("capture()")
    header, encoded = data.split(",", 1)
    img_bytes = b64decode(encoded)
    img = PIL.Image.open(io.BytesIO(img_bytes))
    return img

# ... (rest of the code)

In [ ]:
# Step 6: 이미지 캡처 + 전처리
img = capture_image()
img = img.resize((224, 224))  # Teachable Machine 모델 입력 크기
img_array = np.array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# Step 7: Google Firebase 비공개키 업로드
from google.colab import files
uploaded = files.upload()  # serviceAccountKey.json 파일 업로드

Saving edu-pico-241205-firebase-adminsdk-q2qyt-82ced375fc.json to edu-pico-241205-firebase-adminsdk-q2qyt-82ced375fc.json


In [ ]:
# Step 8: Google Firebase 설정

import firebase_admin
from firebase_admin import credentials, db

# JSON 키 파일명 확인
key_file = list(uploaded.keys())[0]

# Firebase 초기화
cred = credentials.Certificate(key_file)
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://edu-pico-241205-default-rtdb.firebaseio.com/'  # 이 부분 바꿔줘야 함!
})


ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [ ]:
# Step 9: 예측 및 업로드(to Google Firebase)
predictions = model.predict(img_array)
for i, prob in enumerate(predictions[0]):
    print(f"Class {i}: {prob:.4f}")

# 예시: 가장 높은 확률 클래스 구하기
predicted_index = int(np.argmax(predictions[0]))

# Define class_names with your actual class names
class_names = ['Class 0', 'Class 1']  # Replace with your class names

predicted_label = class_names[predicted_index]
confidence = float(predictions[0][predicted_index])

# Firebase에 업로드
ref = db.reference('face_predictions')
ref.set({
    'label': predicted_label,
    'confidence': round(confidence, 4)
})
print("✅ Firebase에 결과 업로드 완료!")

# 🔑 추가: doorOpen 조건 처리
door_open_state = (predicted_index == 0)

# 🔥 doorOpen 태그 업로드
ref_door = db.reference('doorOpen')
ref_door.set(door_open_state)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Class 0: 1.0000
Class 1: 0.0000


NameError: name 'db' is not defined

✅ Firebase에 결과 업로드 완료!
